In [10]:
import os

In [11]:
%pwd

'c:\\Users\\N Ramaraju\\Desktop\\Projects\\LLM-Text-Detection\\research'

In [12]:
os.chdir("../")

In [13]:
%pwd

'c:\\Users\\N Ramaraju\\Desktop\\Projects\\LLM-Text-Detection'

In [14]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    x_test_path: Path
    x_train_path: Path
    y_test_encoded_path: Path
    y_train_encoded_path: Path
    model_name: str
    learning_rate: float
    loss: str
    metrics: list
    target_column: str

In [15]:
from LLM_text_detection.utils.common import *
from LLM_text_detection.constants import *

In [16]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.Simple_nn
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir= config.root_dir,
            x_test_path= config.x_test_path,
            x_train_path= config.x_train_path,
            y_test_encoded_path= config.y_test_encoded_path,
            y_train_encoded_path= config.y_train_encoded_path,
            model_name= config.model_name,
            learning_rate= params.learning_rate,
            loss= params.loss,
            metrics= params.metrics,
            target_column= schema.Label
        )

        return model_trainer_config

In [17]:
import pandas as pd
import os
from  LLM_text_detection import logger
import pickle
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [18]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train_model(self):
        # Load the training and testing data from the CSV files using self.config
        X_train = pd.read_csv(self.config.x_train_path).values
        X_test = pd.read_csv(self.config.x_test_path).values
        y_train_encoded = pd.read_csv(self.config.y_train_encoded_path).values.flatten()
        y_test_encoded = pd.read_csv(self.config.y_test_encoded_path).values.flatten()
        logger.info("Data Loaded")
        # Build a simple neural network model
        model = Sequential([
            Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
            Dropout(0.3),
            Dense(64, activation='relu'),
            Dropout(0.3),
            Dense(1, activation='sigmoid')
        ])
  
        # Compile the model
        model.compile(optimizer=Adam(learning_rate=self.config.learning_rate),
                    loss=self.config.loss,
                    metrics=self.config.metrics)
        logger.info("Model Compiled")              
        
        # Train the model
        history = model.fit(X_train, y_train_encoded,
                            epochs=10,
                            batch_size=32,
                            validation_data=(X_test, y_test_encoded),
                            verbose=1)
        logger.info("Model training Completed")              
        # Save the model
        model.save(os.path.join(self.config.root_dir, self.config.model_name))
        logger.info("Model Saved")      
        return history  

In [19]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train_model()
except Exception as e:
    raise e

[2024-06-22 12:17:09,231: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-06-22 12:17:09,235: INFO: common: yaml file: params.yaml loaded successfully]
[2024-06-22 12:17:09,237: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-06-22 12:17:09,241: INFO: common: created directory at: artifacts]
[2024-06-22 12:17:09,243: INFO: common: created directory at: artifacts/model_trainer]


[2024-06-22 12:17:52,365: INFO: 3583032377: Data Loaded]


c:\Users\N Ramaraju\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


[2024-06-22 12:17:52,860: INFO: 3583032377: Model Compiled]
Epoch 1/10
1157/1157 ━━━━━━━━━━━━━━━━━━━━ 15s 11ms/step - accuracy: 0.9390 - loss: 0.1303 - val_accuracy: 0.9958 - val_loss: 0.0159
Epoch 2/10
1157/1157 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - accuracy: 0.9982 - loss: 0.0062 - val_accuracy: 0.9957 - val_loss: 0.0167
Epoch 3/10
1157/1157 ━━━━━━━━━━━━━━━━━━━━ 12s 10ms/step - accuracy: 0.9995 - loss: 0.0021 - val_accuracy: 0.9954 - val_loss: 0.0188
Epoch 4/10
1157/1157 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - accuracy: 0.9993 - loss: 0.0021 - val_accuracy: 0.9951 - val_loss: 0.0207
Epoch 5/10
1157/1157 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - accuracy: 0.9997 - loss: 9.3309e-04 - val_accuracy: 0.9945 - val_loss: 0.0233
Epoch 6/10
1157/1157 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - accuracy: 0.9998 - loss: 6.8361e-04 - val_accuracy: 0.9947 - val_loss: 0.0263
Epoch 7/10
1157/1157 ━━━━━━━━━━━━━━━━━━━━ 15s 13ms/step - accuracy: 0.9994 - loss: 0.0016 - val_accuracy: 0.9942 - val_loss: 0.0368
Epoch 8/1